In [23]:
import os
import fastf1

In [25]:
os.makedirs("cache", exist_ok=True)
os.makedirs("data", exist_ok=True)

In [27]:
import fastf1
import pandas as pd

fastf1.Cache.enable_cache("cache")

# Carregar sessió
session = fastf1.get_session(2025, "British Grand Prix", "R")
session.load()

# Laps
laps = session.laps
laps = laps[laps["IsAccurate"]]

# Selecció de pilots (exemple)
drivers = ["VER", "NOR", "HAM", "LEC", "PIA", "HUL", "STR", "ALO", "TSU", "SAI", "OCO", "BEA", "ANT", "RUS", "HAD", "LAW", "ALB", "GAS", "COL", "BOR"]
laps_sel = laps[laps["Driver"].isin(drivers)]

# Eliminar outlaps / inlaps
laps_clean = laps_sel[
    (laps_sel["PitOutTime"].isna()) &
    (laps_sel["PitInTime"].isna())
]

# Guardar per visualització
laps_clean.to_csv("data/laps_race.csv", index=False)


core           INFO 	Loading data for British Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '27', '44', '1', '10', '18', '23', '14', '63', '87', '55', '31', '16', '22', '12', '6', '5', '30', '43']


In [28]:
import pandas as pd

laps = pd.read_csv("data/laps_race.csv")

# Convertir LapTime a segons
laps["LapTime"] = pd.to_timedelta(laps["LapTime"])
laps["LapTimeSeconds"] = laps["LapTime"].dt.total_seconds()

consistency = (
    laps
    .groupby("Driver")
    .agg(
        mean_lap_time=("LapTimeSeconds", "mean"),
        std_lap_time=("LapTimeSeconds", "std"),
        laps_completed=("LapNumber", "count")
    )
    .reset_index()
)

consistency["consistency_index"] = (
    consistency["std_lap_time"] / consistency["mean_lap_time"]
)

consistency


,Driver,mean_lap_time,std_lap_time,laps_completed,consistency_index
0,ALB,101.483667,6.608068,33,0.065115
1,ALO,100.790656,6.272151,32,0.062229
2,ANT,109.131250,4.404417,4,0.040359
3,BEA,101.345636,6.303098,33,0.062194
4,GAS,100.917969,5.960112,32,0.059059
5,HAD,106.836250,5.260689,4,0.049241
6,HAM,100.236000,5.893925,32,0.058800
7,HUL,100.309938,5.552993,32,0.055358
8,LEC,101.163576,6.148423,33,0.060777
9,NOR,99.588424,5.339784,33,0.053619


In [29]:
from scipy.stats import linregress
import pandas as pd
import os

laps = pd.read_csv("data/laps_race.csv")
laps["LapTime"] = pd.to_timedelta(laps["LapTime"])
laps["LapTimeSeconds"] = laps["LapTime"].dt.total_seconds()

# Consistència
consistency = (
    laps
    .groupby("Driver")
    .agg(
        mean_lap_time=("LapTimeSeconds", "mean"),
        std_lap_time=("LapTimeSeconds", "std"),
        laps_completed=("LapNumber", "count")
    )
    .reset_index()
)

consistency["consistency_index"] = (
    consistency["std_lap_time"] / consistency["mean_lap_time"]
)

# Degradació
degradation = []

for (driver, stint), df in laps.groupby(["Driver", "Stint"]):
    if len(df) < 5:
        continue

    slope, *_ = linregress(
        df["LapNumber"],
        df["LapTimeSeconds"]
    )

    degradation.append({
        "Driver": driver,
        "Stint": stint,
        "Compound": df["Compound"].iloc[0],
        "degradation_slope": slope
    })

degradation_df = pd.DataFrame(degradation)

# Resum per pilot
metrics_summary = consistency.merge(
    degradation_df.groupby("Driver")["degradation_slope"]
    .mean()
    .reset_index(),
    on="Driver",
    how="left"
)

metrics_summary.to_csv("data/metrics_summary.csv", index=False)

metrics_summary


,Driver,mean_lap_time,std_lap_time,laps_completed,consistency_index,degradation_slope
0,ALB,101.483667,6.608068,33,0.065115,-0.452396
1,ALO,100.790656,6.272151,32,0.062229,-0.691333
2,ANT,109.131250,4.404417,4,0.040359,NaN
3,BEA,101.345636,6.303098,33,0.062194,-0.542982
4,GAS,100.917969,5.960112,32,0.059059,-0.456878
5,HAD,106.836250,5.260689,4,0.049241,NaN
6,HAM,100.236000,5.893925,32,0.058800,-0.472139
7,HUL,100.309938,5.552993,32,0.055358,-0.348953
8,LEC,101.163576,6.148423,33,0.060777,-0.481953
9,NOR,99.588424,5.339784,33,0.053619,-0.185162
